<a href="https://colab.research.google.com/github/GiX007/agent-labs/blob/main/02_building_systems_with_ChatGPT_API/00_chat_format.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chat Format and Tokens

## Setup

#### Load the API key and relevant Python libaries.

First, we load the OpenAI API key (like we did in the previous course).

In [ ]:
import os
import openai
import tiktoken
from dotenv import load_dotenv, find_dotenv
dotenv_path = find_dotenv() or '/content/OPENAI_API_KEY.env' # read local .env file
load_dotenv(dotenv_path)

openai_api_key = os.getenv('OPENAI_API_KEY')
client = openai.OpenAI(api_key=openai_api_key)

#### helper function


This may look familiar if you took the earlier course "ChatGPT Prompt Engineering for Developers" Course.

Throughout this course, we will use OpenAI's `gpt-4o-mini` model and the [chat completions endpoint](https://platform.openai.com/docs/guides/chat).

This helper function will make it easier to use prompts and look at the generated outputs.

In [ ]:
def get_completion(prompt, model="gpt-4o-mini"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

## Prompt the model and get a completion

In [ ]:
response = get_completion("What is the capital of France?")

In [ ]:
print(response)

The capital of France is Paris.


## Tokens

In [ ]:
response = get_completion("Take the letters in lollipop and reverse them")
print(response)

Reversing the letters in "lollipop" gives you "popillol."


The following example is coming from ann older implementation using the model ```gpt-3.5-turbo```. Notice the how the reply differs and how we got the correct answer.

In [ ]:
response = get_completion("Take the letters in lollipop and reverse them")
print(response)

pilpolol


"lollipop" in reverse should be "popillol"

In [ ]:
response = get_completion("""Take the letters in l-o-l-l-i-p-o-p and reverse them""")

In [ ]:
response

'p-o-p-i-l-l-o-l'

## Helper function (chat format)

In [ ]:
def get_completion_from_messages(messages,
                                 model="gpt-4o-mini",
                                 temperature=0,
                                 max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut
    )
    return response.choices[0].message.content

The Effect of Temperature on LLM Output

Temperature is a crucial parameter that controls the degree of randomness and creativity in the model's output. For engineering robust applications, this setting must be carefully chosen.

| Temperature Value | Randomness/Creativity | Effect on Output | Best Use Case |
| :--- | :--- | :--- | :--- |
| **0.0** (or very close to zero) | **Minimal** (Deterministic) | The model always selects the most statistically probable next token. Outputs are factual, concise, and repeatable. | **Factual Q&A, Summarization, Code Generation, Classification, and Agentic Workflows.** |
| **1.0** (or higher) | **Maximal** (High Variability) | The model considers many less-likely tokens, leading to more "surprising" or diverse outputs. The results are less predictable and may occasionally hallucinate. | **Brainstorming, Creative Writing, Story Generation, Ideation.** |

In [ ]:
messages =  [
{'role':'system', # specify the overall tone of llm
 'content':"""You are an assistant who responds in the style of Dr Seuss."""},
{'role':'user', # specific instruction that llm will carry out
 'content':"""write me a very short poem about a happy carrot"""},
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

In the garden, bright and fair,  
Lives a jolly, jolly carrot, with a top of orange hair.  
He giggles with glee, in the sun’s warm glow,  
Wiggling his leaves, as he sways to and fro.  

“I'm crunchy, I'm sweet, oh what a delight!  
A nibble from me makes the veggies feel bright!”  
So if you find a carrot, so happy and spry,  
Give him a hug, let your spirit fly high!


In [ ]:
# length
messages =  [
{'role':'system',
 'content':'All your responses must be one sentence long.'},
{'role':'user',
 'content':'write me a story about a happy carrot'},
]
response = get_completion_from_messages(messages, temperature =1)
print(response)

Once upon a time in a vibrant garden, a happy carrot named Carl danced under the warm sun, dreaming of becoming a delicious salad.


In [ ]:
# combined (style + length)
messages =  [
{'role':'system',
 'content':"""You are an assistant who responds in the style of Dr Seuss. All your responses must be one sentence long."""},
{'role':'user',
 'content':"""write me a story about a happy carrot"""},
]
response = get_completion_from_messages(messages, temperature =1)
print(response)

In a garden so bright, with sunshine and cheer, lived a carrot named Carl who spread joy far and near!


In [ ]:
def get_completion_and_token_count(messages,
                                   model="gpt-4o-mini",
                                   temperature=0,
                                   max_tokens=500):

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    # print(response) # returns a dict

    content = response.choices[0].message.content

    token_dict = {
        'prompt_tokens':response.usage.prompt_tokens,
        'completion_tokens':response.usage.completion_tokens,
        'total_tokens':response.usage.total_tokens,
    }

    return content, token_dict

In [ ]:
messages = [
{'role':'system',
 'content':"""You are an assistant who responds in the style of Dr Seuss."""},
{'role':'user',
 'content':"""write me a very short poem about a happy carrot"""},
]
response, token_dict = get_completion_and_token_count(messages)

In [ ]:
print(response)

In a garden so bright, where the sun shines so fair,  
Lived a jolly young carrot with bright orange hair.  
He danced with the daisies, he twirled with the bees,  
“Oh, life is delightful!” he sang in the breeze.  

With a wiggle and giggle, he’d wiggle all day,  
In the soil so warm, he would laugh and would play.  
For a happy young carrot, with joy in his heart,  
Knows that life in the garden is truly an art!


In [ ]:
print(token_dict)

{'prompt_tokens': 35, 'completion_tokens': 111, 'total_tokens': 146}


#### Notes on using the OpenAI API outside of this classroom

To install the OpenAI Python library:
```
!pip install openai
```

The library needs to be configured with your account's secret key, which is available on the [website](https://platform.openai.com/account/api-keys).

You can either set it as the `OPENAI_API_KEY` environment variable before using the library:
 ```
 !export OPENAI_API_KEY='sk-...'
 ```

Or, set `openai.api_key` to its value (if you only run locally):

```
import openai
openai.api_key = "sk-..."
```